In [1]:
import tweepy
import configparser

In [39]:
file = open('project-data/train.data.txt', 'r')
Lines = file.readlines()
events =[]
# Strips the newline character
for line in Lines:
    events.append(list(map(int,line.strip('\n').split(','))))

In [40]:
config = configparser.ConfigParser()
config.read('config.ini')

consumer_key = config['twitter']['consumer_key']
consumer_secret = config['twitter']['consumer_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']
print(consumer_key)
print(consumer_secret)
print(access_token)
print(access_token_secret)

D36zKRt2oqq38TjnRQZc4zPwh
x2Gke0pktMnOdWliytf4fQqMc2vwAHIyUDKz7TkSBzY0pVaWuc
1517394779260473344-FInfqjNiYmkXqPHzifd5wQ2bM0lZhI
P2tmPTGOa5kH2gRsUOXCDRrfpFNwjtK9PWmPmDkNrUpPJ


In [41]:
# get_tweets only return 100 results, handle the case when there is more than 100
def lookup_tweets(tweet_IDs, client):
    full_tweets = []
    tweet_count = len(tweet_IDs)
    try:
        for i in range(int((tweet_count / 100) + 1)):
            # Catch the last group if it is less than 100 tweets
            end_loc = min((i + 1) * 100, tweet_count)
            full_tweets.extend(
                client.get_tweets(tweet_IDs[i * 100:end_loc],user_auth=True)
            )
        return full_tweets
    except tweepy.errors.TweepyException:
        print('Something went wrong, quitting...')

In [43]:
# authentication
client = tweepy.Client(consumer_key=consumer_key, consumer_secret=consumer_secret,
                                   access_token=access_token, access_token_secret=access_token_secret)



In [46]:
#first 10 events
for event in events[:10]:

    results = lookup_tweets(event, client)

    for event in results:
        if event:
            print(event)

[<Tweet id=1250219300389974016 text='5. Can regularly rinsing your nose with saline help prevent infection with the new coronavirus? https://t.co/ccMjhhD7BK'>, <Tweet id=1250219116993974272 text='4. Can eating garlic help prevent infection with the new coronavirus? #COVID19Malaysia https://t.co/q133xXBiwl'>, <Tweet id=1250219437027766273 text='6. Do vaccines against pneumonia protect you against the new coronavirus? https://t.co/wL0mlEqU95'>, <Tweet id=1250219620939657216 text='7. Can spraying alcohol or chlorine all over your body kill the new coronavirus? #Chamber https://t.co/zunVR7Ht0V'>, <Tweet id=1250219777185873922 text='8. How effective are thermal scanners in detecting people infected with the new coronavirus? https://t.co/nyLOyKAb1H'>, <Tweet id=1250219894429208577 text='9. Can an ultraviolet disinfection lamp kill the new coronavirus? https://t.co/ZrlllbkIjm'>, <Tweet id=1250219998842216448 text='10. Are hand dryers effective in killing the new coronavirus? https://t.co/cSDK

[<Tweet id=524958128392376320 text='Ottawa police confirm that there were multiple suspects in the shooting, saying there were "possibly 3" shooters. http://t.co/In9OgOxYXM'>, <Tweet id=524961934064754688 text="@WSJ @HeyTammyBruce I hope this isn't just a prelude to a bigger attack.">, <Tweet id=524959028061798401 text='@WSJ Killers go berserk when cornered.  Henceforth, no lapses in security should be there.'>, <Tweet id=524959339745140736 text='@45_nabeel hahaha, you said far more than you think'>, <Tweet id=524959070407503873 text='@WSJ @macdonaldajm terrorists in ottawa'>, <Tweet id=525016147448836096 text='@WSJ when Canadians of all people start shooting, well, "c\'est la fin du monde, non?"'>]
[<Tweet id=1239295488677085185 text="if the primary focus of a government isn't to alleviate suffering/death and promote happiness/wellbeing for its citizens, why do we need it to begin with?">]
[{'value': '1239562248990806016', 'detail': 'Could not find tweet with ids: [1239562248990806016